# Hybrid Trust PL PKI

## Setting up the Blockchain

Deploy the smart contract.

In [ ]:
import ehrmann.middleware as mw
import ehrmann.trustGraph as tg

VERBOSE = False

mw = mw.middleware(VERBOSE)

Setting up unlocked CA accounts.

In [ ]:
w3_accounts = mw.get_w3_accounts()

cas = ['ca_1', 'ca_2', 'ca_3', 'ca_4', 'ca_5']
cas = dict(zip(cas, w3_accounts[1:6]))
cas['invalid'] = '0x0000000000000000000000000000000000000000'  # for visualization purposes
cas_reversed = {v: k for k, v in cas.items()}

Generate the hash for google.com.

In [ ]:
google_cert = open("demo-cert/google.com.cer", "rb")
google_cert_hash = mw.hash(google_cert.read())[-20:]
print(google_cert_hash)

Initialize Google Cert Manager.

In [ ]:
google_cert_manager = w3_accounts[9]

# Demonstration -> Fulfillment of Requirements
- Demo 1: Trust network is transparent (Requirement: Transparency)
- Demo 2: CAs can easily add Domains (Requirement: Ease of Use, User Authentication)
- Demo 3: Clients can personalize their trust stores (Requirement: Ease of Use)
- Demo 4: Client validates certificate (Requirement: Ease of Use, Privacy of User's Identity)
- Demo 5: Domain Owner is compromised and wants to revoke a certificate (Requirement: Certificate Revocation)
- Demo 6: CA revokes trust in a domain (Requirement: Certificate Revocation)


## Demo 1: Trust Graph
Build up net of trust.

In [ ]:
mw.ca_trusts_ca(cas['ca_1'], cas['ca_2'])
mw.ca_trusts_ca(cas['ca_2'], cas['ca_3'])
mw.ca_trusts_ca(cas['ca_3'], cas['ca_4'])
mw.ca_trusts_ca(cas['ca_2'], cas['ca_4'])
mw.ca_trusts_ca(cas['ca_4'], cas['ca_5'])
mw.ca_trusts_ca(cas['ca_5'], cas['ca_3'])

Generating graph based on blockchain network.

In [ ]:
G = tg.create_graph(mw, cas, cas_reversed)

Revoking trust of one CA to another.

In [ ]:
mw.ca_revokes_trust_in_ca(cas['ca_2'], cas['ca_3'])

Generating new graph (Every time a CA starts trusting or revokes its trust in another CA, the graph must be rebuild).

In [ ]:
G = tg.create_graph(mw, cas, cas_reversed)

<span style="color:blue">Requirements: Transparency</span>

## Demo 2: Issue of certificates
CAs 4 and 5: `We want to trust Domain 'google.com'`

In [ ]:
domain_signed_cas = ['ca_4', 'ca_5']
for ca in domain_signed_cas:
    mw.add_certificate(cas[ca], google_cert_hash, validity_not_after=1518220800, validity_not_before=1514764800,
                       self_destroy_hash=mw.hash(b'50'))

<span style="color:blue">Requirements: Ease of Use, User Authentication</span>

## Demo 3: Client trust store
Client: `I want to trust 'ca_1' and 'ca_3', and my maximum trustpath length shoud be 2.`

Trust path lenghts:
- length: 0 `I trust nobody`
- length: 1 `I trust my own CA`
- length: 2 `I trust my own and the ones my CA trusts.`

In [ ]:
client_trusted_cas = ['ca_1', 'ca_3']
max_trust_path_length = 2

<span style="color:blue">Requirements: Ease of Use</span>

## Demo 4: Client certificate validation
Client: `I want to visit google.com. Let's see whether the certificate is valid.`

Domain: `You can verify it querying one of my CAs. I'm signed by ca_5 and ca_4!`
1. Getting shortest Path between two CAs.


In [ ]:
shortest_path = tg.ca_in_trust_path_radius(G, client_trusted_cas, domain_signed_cas, max_trust_path_length=max_trust_path_length)
print(shortest_path)

2. Since a path exists, we can query the signing CA whether the certificate is valid.

In [ ]:
if len(shortest_path) > 1:
    mw.retrieve_certificate_status(google_cert_hash, cas[shortest_path[-1]])  # last ca node of trust path

<span style="color:blue">Requirements: Ease of Use, Privacy of User's Identity</span>

## Demo 5: Domain Owner is compromised
Google Cert Manager: `I'm compromised, my private Key got lost/published and I can't reach my CA`

Solution: Self Destruction

In [ ]:
mw.revoke_certificate_as_owner(google_cert_manager, google_cert_hash, cas['ca_4'], b'50')

### Client verification

In [ ]:
mw.retrieve_certificate_status(google_cert_hash, cas['ca_4'])

<span style="color:blue">Requirements: Certificate Revocation</span>

## Demo 6: CA revokes trust
CA 5: `google.com is behaving wiredly. I want to revoke my trust to Domain domain_1 immediately.`

In [ ]:
mw.retrieve_certificate_status(google_cert_hash, cas['ca_5'])

In [ ]:
mw.revoke_certificate_as_ca(cas['ca_5'], google_cert_hash)

### Client verification

In [ ]:
mw.retrieve_certificate_status(google_cert_hash, cas['ca_5'])

<span style="color:blue">Requirements: Certificate Revocation</span>